# Quantum State Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** May 15, 2018 

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.tools.qi.qi import state_fidelity

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 2-Qubit state tomography Example

In [2]:
# Create a state preparation circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell.qasm())

job = qiskit.execute(bell, 'local_statevector_simulator')
psi_bell = job.result().get_data(bell).get('statevector')
print(psi_bell)

OPENQASM 2.0;
include "qelib1.inc";
qreg q0[2];
h q0[0];
cx q0[0],q0[1];

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
# Generate circuits and run on simulator
t = time.time()
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, 'local_qasm_simulator', shots=5000)
print('Time taken:', time.time() - t)

# Extract tomography data so that countns are indexed by measurement configuration
# Note that the None labels are because this is state tomography instead of process tomography
# Process tomography would have the preparation state labels there

tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

Time taken: 0.049649953842163086


{(('X', 'X'), None): {'00': 2513, '11': 2487},
 (('X', 'Y'), None): {'00': 1238, '01': 1296, '10': 1256, '11': 1210},
 (('X', 'Z'), None): {'00': 1215, '01': 1262, '10': 1282, '11': 1241},
 (('Y', 'X'), None): {'00': 1259, '01': 1264, '10': 1267, '11': 1210},
 (('Y', 'Y'), None): {'01': 2491, '10': 2509},
 (('Y', 'Z'), None): {'00': 1254, '01': 1254, '10': 1262, '11': 1230},
 (('Z', 'X'), None): {'00': 1249, '01': 1173, '10': 1298, '11': 1280},
 (('Z', 'Y'), None): {'00': 1283, '01': 1275, '10': 1184, '11': 1258},
 (('Z', 'Z'), None): {'00': 2480, '11': 2520}}

In [4]:
# Generate fitter data and reconstruct density matrix
data_bell, basis_bell = tomo.fitter_data(tomo_counts_bell)
rho_bell = tomo.cvx_fit(data_bell, basis_bell, trace=1)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 0.9999701269657615


## Including Calibration Error (Simulated)

Suppose we include T1 relaxation during measurement, but all other circuit operations are perfect.
Noise parameters:
* measurement time: 5 microseconds
* T_1: 50 microseconds
* Population of excited state: 5%

The assignment fidelity for this can be calculated from
* P(0|0) = 1 - P(1|0)
* P(1|0) = pop1 * exp(- t_meas / t_1)
* P(0|1) = (1 - pop1) * exp(- t_meas / t_1)
* P(1|1) = 1 - P(0|1)

the calibration error matrix is
M = [[P(0|0), P(0|1)], [P(1|0), P(1|1)]]

In [5]:
pop1 = 0.05
T1 = 50
t_meas = 5

noise = {
    'relaxation_rate': 1. / T1,
    'thermal_populations': [1 - pop1, pop1],
    'measure': {'gate_time': t_meas}
}

# Generate circuits and run on simulator
job_cal = qiskit.execute(qst_bell, 'local_qasm_simulator', shots=5000,
                         config={'noise_params': noise})

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_noise = tomo.tomography_data(job_cal.result(), qst_bell)
tomo_counts_noise

{(('X', 'X'), None): {'00': 2520, '01': 210, '10': 240, '11': 2030},
 (('X', 'Y'), None): {'00': 1493, '01': 1271, '10': 1244, '11': 992},
 (('X', 'Z'), None): {'00': 1469, '01': 1219, '10': 1261, '11': 1051},
 (('Y', 'X'), None): {'00': 1501, '01': 1225, '10': 1233, '11': 1041},
 (('Y', 'Y'), None): {'00': 444, '01': 2225, '10': 2304, '11': 27},
 (('Y', 'Z'), None): {'00': 1403, '01': 1283, '10': 1252, '11': 1062},
 (('Z', 'X'), None): {'00': 1500, '01': 1221, '10': 1252, '11': 1027},
 (('Z', 'Y'), None): {'00': 1477, '01': 1231, '10': 1240, '11': 1052},
 (('Z', 'Z'), None): {'00': 2522, '01': 239, '10': 209, '11': 2030}}

In [6]:
# Create Calibration matrix

P1_0 = pop1 * np.exp(-t_meas / T1)
P0_1 = (1 - pop1) * np.exp(-t_meas / T1)

# Single qubit calibration matrix
cal_mat_single = np.array([[1 - P1_0, P0_1], [P1_0, 1 - P0_1]])

# 2-qubit calibration matrix
cal_mat = np.kron(cal_mat_single, cal_mat_single)

In [7]:
# Fitting without calibration matrix
data_noise, basis_noise = tomo.fitter_data(tomo_counts_noise)
rho_noise = tomo.cvx_fit(data_noise, basis_noise, trace=1)
print('Fitting without using calibration data')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise))

Fitting without using calibration data
Fit Fidelity = 0.8629529018407758


In [8]:
# Fitting with calibration matrix by modifying POVMs
data_noise_cal0, basis_noise_cal0 = tomo.fitter_data(tomo_counts_noise,
                                                     calibration_matrix=cal_mat,
                                                     calibration_method=0)
rho_noise_cal0 = tomo.cvx_fit(data_noise_cal0, basis_noise_cal0, trace=1)
print('Using calibration data to update basis matrix')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise_cal0))

Using calibration data to update basis matrix
Fit Fidelity = 0.9976647540380019


In [9]:
# Fitting with calibration matrix by using the pseudo-inverse to update data
data_noise_cal1, basis_noise_cal1 = tomo.fitter_data(tomo_counts_noise,
                                                     calibration_matrix=cal_mat,
                                                     calibration_method=1)
rho_noise_cal1 = tomo.cvx_fit(data_noise_cal1, basis_noise_cal1, trace=1)
print('Using calibration data to update basis matrix')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise_cal1))

Using calibration data to update basis matrix
Fit Fidelity = 1.4009334882000983


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [10]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, 'local_statevector_simulator')
    psi_rand = job.result().get_data(circ).get('statevector')
    
    qst_circs = tomo.state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, 'local_qasm_simulator',
                         shots=shots)
    tomo_counts = tomo.tomography_data(job.result(), qst_circs)
    data, basis = tomo.fitter_data(tomo_counts, beta=0.5)
    
    rho_cvx = tomo.cvx_fit(data, basis, PSD=True, trace=1)
    rho_mle = tomo.mle_fit(data, basis, PSD=True, trace=1)
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (MLE) =', state_fidelity(psi_rand, rho_mle))

In [11]:
for j in range(10):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 2000)

Random single-qubit unitaries: set 0
F fit (CVX) = 0.993139966795
F fit (MLE) = 0.992502267619
Random single-qubit unitaries: set 1
F fit (CVX) = 0.99756570372
F fit (MLE) = 0.992870413411
Random single-qubit unitaries: set 2
F fit (CVX) = 0.996636242508
F fit (MLE) = 0.992683042129
Random single-qubit unitaries: set 3
F fit (CVX) = 0.993695624583
F fit (MLE) = 0.993728670904
Random single-qubit unitaries: set 4
F fit (CVX) = 0.994762227907
F fit (MLE) = 0.99013622977
Random single-qubit unitaries: set 5
F fit (CVX) = 0.995650056729
F fit (MLE) = 0.994889776396
Random single-qubit unitaries: set 6
F fit (CVX) = 0.991585653611
F fit (MLE) = 0.987107366543
Random single-qubit unitaries: set 7
F fit (CVX) = 0.997408996026
F fit (MLE) = 0.994625246208
Random single-qubit unitaries: set 8
F fit (CVX) = 0.999845955216
F fit (MLE) = 0.997743553853
Random single-qubit unitaries: set 9
F fit (CVX) = 0.996075136028
F fit (MLE) = 0.994086361858


## 5-Qubit Bell State

In [9]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, 'local_statevector_simulator')
psi_bell5 = job.result().get_data(bell5).get('statevector')

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = tomo.state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, 'local_qasm_simulator', shots=5000)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_bell5 = tomo.tomography_data(job.result(), qst_bell5)
data_bell5, basis_bell5 = tomo.fitter_data(tomo_counts_bell5)
print('Time taken:', time.time() - t)

Time taken: 15.38872480392456


In [16]:
t = time.time()
rho_mle_bell5 = tomo.mle_fit(data_bell5, basis_bell5,
                             PSD=True, trace=1)
print('MLE Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_mle_bell5))

MLE Reconstruction
Time taken: 4.590576887130737
Fit Fidelity: 0.994050438135


In [17]:
t = time.time()
rho_sdp_bell5 = tomo.cvx_fit(data_bell5, basis_bell5, PSD=True, solver='CVXOPT')
print('SDP Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_sdp_bell5))

SDP Reconstruction
Time taken: 90.71169567108154
Fidelity: 0.996387710362
